In [ ]:
!pip3 install transformers sentencepiece hazm clean-text[gpl]


In [ ]:
!gdown 1D3yt99D0GcCRCbdKbUQGxbqjkeh91hTg

In [ ]:
!unrar x hamshahri.rar
!cp /content/hamshahriold/Corpus/Hamshahri-Categories.txt /content/
!unzip /content/hamshahriold/Corpus/Hamshahri-Corpus.zip
!unzip /content/hamshahriold/Corpus/PersianStopWords.zip

In [ ]:
import pandas as pd
import hazm
from transformers import AutoConfig, AutoTokenizer, TFAutoModel, AutoModel, DataCollatorWithPadding

# Save data to csv file

In [ ]:
# [[DID value, Date value, CAT, text]]
corpus = []
tmp_text = " "
tmp_values = []
c = 0
with open('Hamshahri-Corpus.txt', "rb") as file:
  for line in file:
    line = line.decode("UTF-8")
    if ".DID" in line:
      if len(tmp_text.split(' ')) < 1000:
        tmp_values.append(tmp_text)
        corpus.append(tmp_values)
      tmp_text = ""
      tmp_values = []
      tmp_values.append(line.replace(".DID\t", "").replace("\r\n",""))
    elif ".Date" in line:
      tmp_values.append(line.replace(".Date\t", "").replace("\r\n","").replace("\\", "/"))
    elif ".Cat" in line:
      tmp_values.append(line.replace(".Cat\t", "").replace("\r\n",""))
    else:
      tmp_text += (line.strip() + " ")
corpus.pop(0)
len(corpus)

In [ ]:
df = pd.DataFrame(corpus, columns=['DID', 'Date', 'Cat', 'Text'])
df

In [ ]:
df.to_csv("dataset.csv", date_format='%Y%m%d')

#preprocessing

In [ ]:
df = df[['Text', 'Cat']]

In [ ]:
x = 0
y = 0
z = 0
for txt in df["Text"]:
  l = len(txt.split(" "))
  if len(txt.split(" ")) < 10:
    z += 1
    x = l
    y=txt
x
z

In [ ]:
y

In [ ]:
# stop word
stop_words_list = []
with open('PersianStopWords.txt', "rb") as file:
  for line in file:
    stop_words_list.append(line.decode("UTF-8").replace('\r\n', ""))

for idx, txt in enumerate(df["Text"]):
  word_tokenized =  hazm.word_tokenize(txt)
  cps = ""
  for word in word_tokenized:
    if word not in stop_words_list:
      cps += word + " "

  df.loc[idx].at['Text'] = cps
  if idx % 50000 == 0:
    print(idx, "numbers cleaned")

### Normalization

In [ ]:
df['text_len_by_words'] = df['Text'].apply(lambda t: len(hazm.word_tokenize(t)))
min_max_len = df["text_len_by_words"].min(), df["text_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

In [ ]:
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='text_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [ ]:
minlim, maxlim = 10, 500
data_gl_than(df, maxlim, minlim)

In [ ]:
# remove comments with the length of fewer than three words
df['text_len_by_words'] = df['text_len_by_words'].apply(lambda len_t: len_t if minlim <= len_t <= maxlim else None)
df = df.dropna(subset=['text_len_by_words'])
df = df.reset_index(drop=True)

In [ ]:
min_max_len = df["text_len_by_words"].min(), df["text_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

In [ ]:
unique_cats = list(sorted(df['Cat'].unique()))
print(f'We have #{len(unique_cats)}: {unique_cats}')

#Model loading

In [ ]:
model_repo = "HooshvareLab/bert-fa-base-uncased-clf-persiannews"
config = AutoConfig.from_pretrained(model_repo)
tokenizer = AutoTokenizer.from_pretrained(model_repo)
model = AutoModel.from_pretrained(model_repo, num_labels=len(unique_cats))

In [ ]:
model.config.max_length = 20

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples, truncation=True)
cf["Text"].map(preprocess_function)

In [ ]:
len(tokenizer(cf["Text"][10], truncation=True, padding=True)['input_ids'])

In [ ]:
cf["Text"][0]

In [80]:
import pandas as pd
import hazm
from transformers import AutoConfig, AutoTokenizer, TFAutoModel, AutoModel, DataCollatorWithPadding

# Save data to csv file

In [17]:
# [[DID value, Date value, CAT, text]]
corpus = []
tmp_text = " "
tmp_values = []
c = 0
with open('Hamshahri-Corpus.txt', "rb") as file:
  for line in file:
    line = line.decode("UTF-8")
    if ".DID" in line:
      if len(tmp_text.split(' ')) < 1000:
        tmp_values.append(tmp_text)
        corpus.append(tmp_values)
      tmp_text = ""
      tmp_values = []
      tmp_values.append(line.replace(".DID\t", "").replace("\r\n",""))
    elif ".Date" in line:
      tmp_values.append(line.replace(".Date\t", "").replace("\r\n","").replace("\\", "/"))
    elif ".Cat" in line:
      tmp_values.append(line.replace(".Cat\t", "").replace("\r\n",""))
    else:
      tmp_text += (line.strip() + " ")
corpus.pop(0)
len(corpus)

150505

In [18]:
df = pd.DataFrame(corpus, columns=['DID', 'Date', 'Cat', 'Text'])
df

,DID,Date,Cat,Text
0,1S1,75/04/02,adabh,جاودانگي در زندگي گروهي از طريق هنر نگاهي به ن...
1,2S1,75/04/02,adabh,رويدادهاي هنري جهان نمايشگاه هنر در خدمت ديكتا...
2,3S1,75/04/02,adabh,برديوار نگارخانه ها گالري گلستان: نمايشگاه طرح...
3,5S1,75/04/02,elmfa,تخته سياه و غباري كه سترده نمي شود... اشاره; ب...
4,6S1,75/04/02,elmfa,احتمال اعتصاب آموزگاران در جمهوري آذربايجان به...
...,...,...,...,...
150500,60055S1,81/11/20,vrzsh,گره هاي كور كشتي باز مي؟ شود گروه ورزشي: با حض...
150501,60055S2,81/11/20,vrzsh,نماينده فدراسيون جهاني واليبال از ايران هر نظر...
150502,60055S3,81/11/20,vrzsh,شكست نامداران تكواندودر پيكارهاي برتر ليگ گروه...
150503,60055S4,81/11/20,vrzsh,ورزشگاه بزرگ دانشگاه آزاد در تهران ساخته مي شو...


In [7]:
df.to_csv("dataset.csv", date_format='%Y%m%d')

#preprocessing

In [19]:
df = df[['Text', 'Cat']]

In [57]:
x = 0
y = 0
z = 0
for txt in df["Text"]:
  l = len(txt.split(" "))
  if len(txt.split(" ")) < 10:
    z += 1
    x = l
    y=txt
x
z

2358

In [53]:
y

'فيلمبرداري شده_است گفتني نسخه DVD فيلم جشنواره نمايش داده '

In [60]:
# stop word
stop_words_list = []
with open('PersianStopWords.txt', "rb") as file:
  for line in file:
    stop_words_list.append(line.decode("UTF-8").replace('\r\n', ""))

for idx, txt in enumerate(df["Text"]):
  word_tokenized =  hazm.word_tokenize(txt)
  cps = ""
  for word in word_tokenized:
    if word not in stop_words_list:
      cps += word + " "
      
  df.loc[idx].at['Text'] = cps
  if idx % 50000 == 0:
    print(idx, "numbers cleaned")

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_values(loc, value)


0 numbers cleaned
10000 numbers cleaned
20000 numbers cleaned
30000 numbers cleaned
40000 numbers cleaned
50000 numbers cleaned
60000 numbers cleaned
70000 numbers cleaned
80000 numbers cleaned
90000 numbers cleaned
100000 numbers cleaned
110000 numbers cleaned
120000 numbers cleaned
130000 numbers cleaned
140000 numbers cleaned
150000 numbers cleaned


### Normalization

In [67]:
df['text_len_by_words'] = df['Text'].apply(lambda t: len(hazm.word_tokenize(t)))
min_max_len = df["text_len_by_words"].min(), df["text_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

In [69]:
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='text_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [82]:
minlim, maxlim = 10, 500
data_gl_than(df, maxlim, minlim)

Texts with word length of greater than 11 and less than 500 includes 99.89% of the whole!


In [73]:
# remove comments with the length of fewer than three words
df['text_len_by_words'] = df['text_len_by_words'].apply(lambda len_t: len_t if minlim <= len_t <= maxlim else None)
df = df.dropna(subset=['text_len_by_words'])
df = df.reset_index(drop=True)

In [81]:
min_max_len = df["text_len_by_words"].min(), df["text_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 11.0 	Max: 500.0


In [83]:
unique_cats = list(sorted(df['Cat'].unique()))
print(f'We have #{len(unique_cats)}: {unique_cats}')

We have #104: ['Adabh', 'Aeqts', 'Akhar', 'Busiw', 'Cinew', 'Cultw', 'Econo', 'Eqtes', 'Globa', 'Gozar', 'Kharj', 'Lifew', 'Musical', 'Newsp', 'Nnaft', 'Santj', 'Sciew', 'Shari', 'Siasi', 'Sporw', 'Theatre', 'Thought', 'abksh', 'adabh', 'adarman', 'aeqts', 'akhar', 'art', 'artw', 'axrooz', 'bankb', 'bazar', 'books', 'busiw', 'cartoon', 'cinama', 'cinew', 'city', 'cultw', 'donya', 'earth', 'econo', 'econw', 'ejtem', 'elmfa', 'elmif', 'eqtes', 'eqtsj', 'ertebat', 'erteg', 'gards', 'globa', 'gofgu', 'goftg', 'gozar', 'gqarn', 'gungn', 'hamln', 'havad', 'igozar', 'ikabar', 'imaqal', 'infor', 'intep', 'jjahn', 'jvarz', 'kharj', 'lastp', 'lifew', 'lite', 'maqal', 'maref', 'media', 'mohit', 'mskan', 'musical', 'nameh', 'newsp', 'nnaft', 'norooz', 'polig', 'sanat', 'santj', 'scien', 'sciew', 'shahr', 'shahz', 'shari', 'shora', 'shrst', 'siasi', 'socie', 'soxan', 'sport', 'sporw', 'techn', 'telfn', 'theatre', 'thought', 'vrzsh', 'women', 'ydsht', 'youth', 'zanan']


#Model loading

In [23]:
model_repo = "HooshvareLab/bert-fa-base-uncased-clf-persiannews"
config = AutoConfig.from_pretrained(model_repo)
tokenizer = AutoTokenizer.from_pretrained(model_repo)
model = AutoModel.from_pretrained(model_repo, num_labels=len(unique_cats))

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased-clf-persiannews were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
model.config.max_length = 20

In [41]:
def preprocess_function(examples):
    return tokenizer(examples, truncation=True)
cf["Text"].map(preprocess_function)

TypeError: ignored

In [51]:
len(tokenizer(cf["Text"][10], truncation=True, padding=True)['input_ids'])

221

In [40]:
cf["Text"][0]

'جاودانگي زندگي گروهي طريق هنر نگاهي نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان باطن ظاهري انسانهاهستند هيبتي حشره تابلوهاي نقاشي نمايشگر گوشه زندگي مورچه ديده ايم سنگين خودرا دوش كشد راه خسته نتيجه آمدن حادثه اتفاقي دستش رها گريز خطر گذارد فرار ساعتي نقطه برمي گردد دوش كشيده نفس زنان عرق ريزان منزل رساند داند راه رفت مورچه استعداد غريزه عملي شروع پايان رساند داند نسل آينده مزاياي استفاده نمايد داستان درازي انتها پاياني ميليونها سال گذشته ميلياردها سال خواهد_گذشت وهمين برنامه ميليونها تجديد گفت سير حيات تكامل پايان ناپذير اسرار جهان پاياني بخواهيم چيز پايان ناپذير برسيم انتهايي نيست جايي رسيم كتاب مورچگان نوشته موريس مترلينگ مورچگان تابلوهاي نقاشي احمد طباطبايي جايگاه ويژه دارند نمايشگاهي آثار هنرمند اول 23 خرداد نگارخانه برگ برپا 36 اثر موضوع نمايش عمومي درآمد طباطبايي مدت 18 سالي نقاشي پردازد موضوع مورچگان كار علت انتخاب سوژه گويد روزهاي زندگي غمگين بودم زماني خواب رفتم مورچه ديدم تصميم گرفتم طرح بكشم كار نظرم جالب آمد روز طراحي ادامه دادم نقاشي مدلها تغييراتي ايجاد